In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk

def deteksi_pisang(image_path):
    image = cv2.imread(image_path)
    original_image = image.copy()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([15, 70, 70])    
    upper_yellow = np.array([35, 255, 255])  
    
    lower_green = np.array([35, 40, 40])    
    upper_green = np.array([90, 255, 255])  

    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    kernel = np.ones((5, 5), np.uint8)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel)
    
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)

    yellow_pixels = cv2.countNonZero(mask_yellow)
    green_pixels = cv2.countNonZero(mask_green)
    
    yellow_part = cv2.bitwise_and(image, image, mask=mask_yellow)
    green_part = cv2.bitwise_and(image, image, mask=mask_green)
    
    processed_image = original_image.copy()
    
    yellow_overlay = np.zeros_like(original_image)
    yellow_overlay[mask_yellow > 0] = [0, 255, 255]  
    processed_image = cv2.addWeighted(processed_image, 1, yellow_overlay, 0.3, 0)
    
    green_overlay = np.zeros_like(original_image)
    green_overlay[mask_green > 0] = [0, 255, 0]  
    processed_image = cv2.addWeighted(processed_image, 1, green_overlay, 0.3, 0)
    
    total_detected = yellow_pixels + green_pixels
    yellow_percent = 0
    green_percent = 0
    
    if total_detected > 0:
        yellow_percent = round((yellow_pixels / total_detected) * 100, 2)
        green_percent = round((green_pixels / total_detected) * 100, 2)
    
    if yellow_pixels > green_pixels:
        hasil = f"Pisang Matang\nWarna Kuning: {yellow_pixels} pixel ({yellow_percent}%)\nWarna Hijau: {green_pixels} pixel ({green_percent}%)"
    elif green_pixels > yellow_pixels:
        hasil = f"Pisang Mentah\nWarna Kuning: {yellow_pixels} pixel ({yellow_percent}%)\nWarna Hijau: {green_pixels} pixel ({green_percent}%)"
    else:
        hasil = "Tidak dapat menentukan"
    
    return original_image, processed_image, hasil

def resize_image_proportionally(image, max_width=400, max_height=300):
    width, height = image.shape[1], image.shape[0]
    
    scale_width = max_width / width
    scale_height = max_height / height
    scale = min(scale_width, scale_height)
    
    if scale < 1: 
        new_width = int(width * scale)
        new_height = int(height * scale)
        resized = cv2.resize(image, (new_width, new_height))
        return resized
    
    return image 

def pilih_gambar():
    global image_path
    image_path = filedialog.askopenfilename(title="Pilih Gambar Pisang", filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.webp")])

    if image_path:
        image = cv2.imread(image_path)
        if image is None:
            messagebox.showerror("Error", "Tidak dapat membaca gambar. Coba format gambar lain.")
            return
            
        display_image(image, label_original_image)
        
        btn_proses.config(state=tk.NORMAL)
        
        label_processed_image.config(image='')
        label_hasil.config(text="Klik 'Mulai Proses Deteksi' untuk analisis")
    else:
        messagebox.showwarning("Peringatan", "Tidak ada gambar yang dipilih.")

def display_image(cv_image, label_widget):
    resized_image = resize_image_proportionally(cv_image)
    rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    tk_image = ImageTk.PhotoImage(pil_image)
    
    label_widget.config(image=tk_image)
    label_widget.image = tk_image 

def proses_deteksi():
    if 'image_path' in globals():
        try:
            original_image, processed_image, hasil = deteksi_pisang(image_path)
            display_image(processed_image, label_processed_image)
            
            label_hasil.config(text=hasil)
        except Exception as e:
            messagebox.showerror("Error", f"Terjadi kesalahan saat memproses: {str(e)}")
    else:
        messagebox.showwarning("Peringatan", "Silakan pilih gambar terlebih dahulu.")

root = tk.Tk()
root.title("Deteksi Kematangan Pisang")
root.geometry("900x700") 

frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

frame_images = tk.Frame(root)
frame_images.pack(padx=10, pady=10)

frame_original = tk.Frame(frame_images)
frame_original.pack(side=tk.LEFT, padx=10)

frame_processed = tk.Frame(frame_images)
frame_processed.pack(side=tk.RIGHT, padx=10)

btn_pilih_gambar = tk.Button(frame_buttons, text="Pilih Gambar Pisang", command=pilih_gambar)
btn_pilih_gambar.pack(side=tk.LEFT, padx=5)

btn_proses = tk.Button(frame_buttons, text="Mulai Proses Deteksi", command=proses_deteksi, state=tk.DISABLED)
btn_proses.pack(side=tk.LEFT, padx=5)

tk.Label(frame_original, text="Gambar Asli").pack()
label_original_image = tk.Label(frame_original, bg="lightgray")
label_original_image.pack(padx=5, pady=5)

tk.Label(frame_processed, text="Hasil Deteksi").pack()
label_processed_image = tk.Label(frame_processed, bg="lightgray")
label_processed_image.pack(padx=5, pady=5)

label_hasil = tk.Label(root, text="Pilih gambar terlebih dahulu", font=("Arial", 12), justify=tk.LEFT)
label_hasil.pack(pady=20)

root.mainloop()